<a href="https://colab.research.google.com/github/oulla898/FPGA-based-PS-2-decoder/blob/main/OL-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rasterio geopandas earthengine-api geemap geojson

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='encoded-aspect-442319-r5')

In [ ]:
import ee
import geemap
import json
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# # Initialize Earth Engine
# ee.Initialize()

In [ ]:
def load_geojson(file_path):
    """
    Load GeoJSON file with land use polygons

    Args:
        file_path (str): Path to land_use_polygons.geojson

    Returns:
        dict: Processed land use polygon features
    """
    with open(file_path, 'r') as f:
        geojson_data = json.load(f)

    # Categorize polygons by class
    land_use_classes = {
        'commercial': [],
        'residential': [],
        'industrial': []
    }

    for feature in tqdm(geojson_data['features'], desc="Loading GeoJSON"):
        class_name = feature['properties']['class']
        coordinates = feature['geometry']['coordinates'][0]

        # Convert coordinates to Earth Engine geometry
        ee_geometry = ee.Geometry.Polygon(coordinates)

        if class_name in land_use_classes:
            land_use_classes[class_name].append(ee_geometry)

    return land_use_classes

def get_muscat_satellite_image(start_date='2022-01-01',
                                end_date='2022-12-31'):
    """
    Retrieve cloud-free satellite imagery for Muscat region

    Args:
        start_date (str): Start date for image collection
        end_date (str): End date for image collection

    Returns:
        ee.Image: Preprocessed satellite image
    """
    # Muscat region approximate coordinates
    muscat_bbox = ee.Geometry.Rectangle([58.3, 23.5, 58.6, 23.7])

    # Use corrected Sentinel-2 Surface Reflectance collection
    sentinel_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(muscat_bbox) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

    # Select median composite image
    muscat_image = sentinel_collection.median() \
        .clip(muscat_bbox) \
        .select(['B4', 'B3', 'B2'])  # Red, Green, Blue bands

    return muscat_image

def create_training_data(satellite_image, land_use_classes, patch_size=64):
    """
    Create training data from satellite image and land use classes

    Args:
        satellite_image (ee.Image): Satellite image to sample from
        land_use_classes (dict): Dictionary of land use class geometries
        patch_size (int): Size of image patches

    Returns:
        np.array: Array of image patches
        np.array: Array of labels corresponding to image patches
    """
    image_patches = []
    labels = []
    label_mapping = {'commercial': 0, 'residential': 1, 'industrial': 2}

    for class_name, geometries in land_use_classes.items():
        for geom in tqdm(geometries, desc=f"Sampling {class_name} patches"):
            # Sample patches from the image within the polygon
            # Instead of width and height, use the geometry (geom) to define the region size
            patches = satellite_image.sampleRectangle(region=geom,
                                                       defaultValue=0)
                                                       # removed width and height
            # Convert patch to numpy array
            patch_array = np.array(patches.getInfo()['properties'])
            image_patches.append(patch_array)
            labels.append(label_mapping[class_name])

    return np.array(image_patches), np.array(labels)

def build_model(input_shape):
    """
    Build and compile a CNN model

    Args:
        input_shape (tuple): Shape of input data

    Returns:
        tf.keras.Model: Compiled CNN model
    """
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def main():
    # Load land use polygons
    land_use_path = 'land_use_polygons.geojson'
    land_use_classes = load_geojson(land_use_path)

    # Get satellite imagery
    satellite_image = get_muscat_satellite_image()

    # Create training data
    image_patches, labels = create_training_data(satellite_image, land_use_classes)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(image_patches, labels, test_size=0.2, random_state=42)

    # Build model
    input_shape = (64, 64, 3)  # Assuming patches are 64x64 with 3 channels (RGB)
    model = build_model(input_shape)

    # Train model with progress bar
    for epoch in range(10):
        with tqdm(total=len(X_train), desc=f"Training Epoch {epoch+1}/10") as pbar:
            model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test), callbacks=[TQDMProgressCallback(pbar)])

    # Evaluate model
    with tqdm(total=len(X_test), desc="Evaluating Model") as pbar:
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"Test accuracy: {accuracy:.4f}")

if __name__ == '__main__':
    main()

class TQDMProgressCallback(tf.keras.callbacks.Callback):
    def __init__(self, pbar):
        self.pbar = pbar

    def on_batch_end(self, batch, logs=None):
        self.pbar.update(1)
        self.pbar.set_postfix(loss=logs.get('loss'), accuracy=logs.get('accuracy'))

# Additional setup instructions
print("""
Setup Requirements:
1. Install required libraries:
   pip install earthengine-api geemap numpy matplotlib tensorflow tqdm

2. Authenticate Google Earth Engine:
   ee.Authenticate()

3. Ensure your GeoJSON file is in the correct projection
   (EPSG:3857 or similar Web Mercator)
""")


Sampling commercial patches:   3%|▎         | 126/3988 [00:27<14:03,  4.58it/s]


EEException: Image.sampleRectangle: Too many pixels in sample; must be <= 262144. Got 478992.